# Project Data Type

Objectif : trier des données textuelles par type.  
Les types possibles sont :  
2 : code postal  
3 : coordonnées GPS  
4 : adresse  
5 : numéro SIRET  
6 : Année  
7 : Date  
8 : SIREN  
9 : Code NAF/APE  
10 : Autre  
11 : Téléphone  

In [47]:
# Pour manipuler et analyser des données sous formes de tableaux
import pandas as pd
# Pour importer les fichiers CSV, on utilise glob
import glob

In [48]:
# On récupère tous les CSV situés dans le répertoire "./data" du Jupyter notebook
files = glob.glob("./data/*.csv")

# On visualise les colonnes du jeu de données pour voir un peu la tête que ça a
for file in files:
    df = pd.read_csv(file, sep=";")
    df.info()
    # Séparateur pour mieux visualiser les changements de CSV
    print("---------")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13194 entries, 0 to 13193
Data columns (total 29 columns):
Dispositif                                                                   13194 non-null object
Type de structure                                                            13194 non-null object
Année d'agrément                                                             13194 non-null object
Localisation                                                                 10167 non-null object
Début d'agrément                                                             13194 non-null int64
Fin d'agrément                                                               13194 non-null int64
Numéro SIREN                                                                 10107 non-null float64
Nom de l'entreprise                                                          13194 non-null object
Sigle                                                                        4021 non-null object
Ac

In [52]:
# Ici, je vérifie manuellement de quel type semble être la donnée (exemple ici pour le CSV 2)
df = pd.read_csv(files[1], sep=";")
df.sample(5)

,ID,Date,Axe,Départ,Arrivée,Nombre de trains programmés,Nombre de trains ayant circulé,Nombre de trains annulés,Nombre de trains en retard à l'arrivée,Taux de régularité,Nombre de trains à l'heure pour un train en retard
390,INT_3_A,2014-03,Est Centre,BÉZIERS,CLERMONT FERRAND,31.0,31.0,0.0,0.0,100.0,NaN
2032,INT_23_A,2017-01,Est Centre,PARIS EST,TROYES,216.0,214.0,2.0,36.0,83.2,4.9
343,INT_9_A,2014-02,Nord-Normandie,CAEN,PARIS ST LAZARE,176.0,176.0,0.0,15.0,91.5,10.7
1703,INT_23_B,2017-07,Est Centre,TROYES,PARIS EST,238.0,235.0,3.0,29.0,87.7,7.1
387,INT_20_B,2014-02,Est Centre,PARIS BERCY,NEVERS,135.0,134.0,1.0,18.0,86.6,6.4


Dans cet exemple, je ne suis intéressé que par la colone date globalement

In [43]:
# On crée un pseudo-switch qui permet de connaître le type de la donnée en fonction du nom de la colonne du CSV
# Je procède via un grand nombre de "elif" car il n'y a pas de switch/case en Python
# Et que je galère pas mal avec les dictionnaires par manque d'habitude
def returnType(colName):
    # Code postaux
    if colName in ["Code commune (France)"]:
        return 2
    # Coordonnées GPS
    elif colName in ["Géolocalisation", "coord"]:
        return 3
    # Adresses
    elif colName in ["L4_NORMALISEE", "L4_NORMALISEE"]:
        return 4
    # Numéros SIRET
    elif colName in ["SIRETPS"]:
        return 5
    # Années
    elif colName in ["Début d'agrément", "Fin d'agrément"]:
        return 6
    # Dates
    elif colName in ["Date", "JOUR"]:
        return 7
    # Numéros SIREN
    elif colName in ["Numéro SIREN", "SIREN"]:
        return 8
    # Codes NAF/APE
    elif colName in ["Base SIRENE : Code APE de l'établissement siège"]:
        return 9
    # Téléphones
    elif colName in ["TEL", "Téléphone"]:
        return 11
    # Defaut
    else:
        return 0

In [62]:
# On crée une liste temporaire destinée à recevoir les contenus
tempList = []

# Pour chaque CSV, on crée une DataFrame
for file in files:
    print("iterating on file " + file)
    df = pd.read_csv(file, sep=";")
    # Pour chaque colonne du DataFrame actuellement parcouru
    for col in df.columns:
        # On check si la colonne renvoi un type attendu (ou 0 par défaut)
        if returnType(col) != 0:
            print("processing column " + col)
            # Si le type est attendu, on va récupérer tout le contenu de la colonne ainsi que l'indice de type correspondant
            indice = returnType(col)
            valeurs = df[col].tolist()
            # Pour chaque valeur, on va imprimer dans la liste temporaire [indice, valeur]
            for value in values:
                tempList.append([key, value])

# Lorsque tous les CSV sont traités, on transforme la liste temporaire en DataFrame
dfsum = pd.DataFrame(tempList, columns=["y", "X"])

iterating on file ./data\fr-esr-cir-et-cii-organismes-et-bureaux-de-style-agrees.csv
processing column Début d'agrément
processing column Fin d'agrément
processing column Numéro SIREN
processing column Téléphone
processing column Géolocalisation
processing column Code commune (France)
processing column Base SIRENE : Code APE de l'établissement siège
iterating on file ./data\regularite-mensuelle-intercites.csv
processing column Date
iterating on file ./data\sirc-17804_9075_61171_2018102_E_Q_20180413_015943916.csv
processing column SIREN
processing column L4_NORMALISEE
processing column SIRETPS
processing column TEL
iterating on file ./data\telecom_qualite_services_mobiles_07-2016.csv
processing column coord
iterating on file ./data\validations-sur-le-reseau-de-surface-nombre-de-validations-par-jour-1er-semestr0.csv
processing column JOUR


In [74]:
dfsum.sample(5)

,y,X
33648,7,2017-06
3927,7,2015-06
16116,7,2016-12
34652,7,2015-09
12559,7,2016-04
